In [6]:
import mimetypes
import pathlib
import shutil
import tempfile

from PIL import Image, ImageOps
from replicate.client import Client
from decouple import Config, RepositoryEnv, config
# from  pillow_heif import register_heif_opener.  # for mac images
# register_heif_opener()



In [7]:
# pathlib.Path(__file__).resolve() # For python file
NBS_DIR = pathlib.Path().resolve()
REPO_DIR = NBS_DIR.parent
DATA_DIR = REPO_DIR / 'data'
# INPUT_DIR = DATA_DIR / 'inputs'
# READY_DIR = DATA_DIR / 'ready'
# OUTPUT_DIR = DATA_DIR / 'outputs'
GENERATED_DIR = DATA_DIR / 'generated'
GENERATED_DIR.mkdir(exist_ok=True, parents=True)
REFERENCES_DIR = DATA_DIR / 'references'
REFERENCES_DIR.mkdir(exist_ok=True, parents=True)


config_env = Config(RepositoryEnv(REPO_DIR / '.env'))
REPLICATE_API_TOKEN = config_env('REPLICATE_API_TOKEN', default=None, cast=str)
REPLICATE_MODEL = config('REPLICATE_MODEL', default="slackroo/superme-sumanth-v1")
REPLICATE_MODEL_VERSION = config('REPLICATE_MODEL_VERSION', default="535caef9907be746cc55ccde50aa5eee85d8f987d11c8c4735d70a18d1e37f49", cast=str)

REPLICATE_MODEL

'slackroo/superme-sumanth-v1'

# Generate Image


In [8]:

replicate_client = Client(api_token=REPLICATE_API_TOKEN)

In [9]:
# reference_image_path = REFERENCES_DIR/ "spider-man.webp"
# ref_image = open(reference_image_path, "rb")


In [38]:
model_f_version = f"{REPLICATE_MODEL}:{REPLICATE_MODEL_VERSION}"
prompt = "a phot of TOK who is a architecture explaining building plans to a contractor"
num_outputs = 2
output_format = "jpg"

input_args = {
    "prompt": prompt,
    "num_outputs": 2,
    "output_format": "jpg",
}


In [39]:
replicate_model = replicate_client.models.get(REPLICATE_MODEL)
replicate_version = replicate_model.versions.get(REPLICATE_MODEL_VERSION)
pred = replicate_client.predictions.create(
    version=replicate_version,
    input=input_args,
)

In [40]:
pred.id

't7zrzh5qr5rma0cp57sr59knjw'

In [47]:
pred.status

'starting'

In [48]:
# upstash -> qstash to jelp with polling process

In [49]:
pred_id = "t7zrzh5qr5rma0cp57sr59knjw"
pred_lookup = replicate_client.predictions.get(pred_id)

In [50]:
pred_lookup.status

'succeeded'

In [51]:

pred.output

In [52]:
pred_urls = pred_lookup.output
pred_urls

['https://replicate.delivery/xezq/wlY6v49qMaKTEB7TYZNTDuWEslCPLhXesSCsQlDdYxZRaARKA/out-0.jpg',
 'https://replicate.delivery/xezq/pXMeFWEsyYWnPSRHDzJPj02HCRra7Bb4yFoINPVAGzERaARKA/out-1.jpg']

In [53]:
import httpx
import random

In [54]:

session_id = random.randint(1, 50_000)
with httpx.Client() as client:

    for i , url in enumerate(pred_urls):
        fname= f"{i}-{session_id}.jpg"
        outpath = GENERATED_DIR / fname
        res =  client.get(url)
        res.raise_for_status()
        with open(outpath, 'wb') as f:
            f.write(res.content)
